# The Matplotlib Jupyter Widget Backend

Enabling interaction with matplotlib charts in the Jupyter notebook and JupyterLab

https://github.com/matplotlib/jupyter-matplotlib

In [34]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import hsv_to_rgb
from math import atan2, hypot, degrees
from ipywidgets import Output
from jpy_video import Video
from os import getenv
import IPython
import ipywidgets as widgets
import json
import boto3

client = boto3.client(
    's3',
    aws_access_key_id=getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=getenv("AWS_SECRET_ACCESS_KEY")
)

with open('./version.json') as f:
    version = json.load(f)


# f = "./examples/source.mp4"

txt = """
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""

IPython.display.display(IPython.display.HTML(data=txt))

upload = widgets.FileUpload(multiple = True)

def handle_upload(change):
    vids = ['mov', 'mp4', 'flv', 'wmv']
    for i in range(0, len(upload.value)):
        f = list(upload.value.values())[i]
        name = str(f["metadata"]["name"])
        content = f["content"]

        if (name.split('.')[1] == 'csv'):
            # csv_name = "./preds" + str(version["v"]) + ".csv"
            csv_name = "./preds.csv"
            with open(csv_name, "w+b") as i:
                i.write(content)            
            with open(csv_name, "rb") as f:
                client.upload_fileobj(f, "saltlab-storage", "preds.csv")

        if (name.split('.')[1] in vids):
            # vid_name = "./up" + str(version["v"]) + ".mp4"
            vid_name = "./up.mp4"
            # print(vid_name)
            with open(vid_name, "w+b") as i:
                i.write(content)
            with open(vid_name, "rb") as f:
                client.upload_fileobj(f, "saltlab-storage", "up.mp4")
    # version["v"]  = version["v"] + 1

    # with open("./version.json", "w") as f:
    #     json.dump(version, f)

upload.observe(handle_upload, names='value')

# Video and Graph

In [35]:
output = Output()
button = widgets.Button(description='Generate Graph')

def on_button_clicked(_):
    with output:
      # "linking function with output"
#       with out:
#           # what happens when we press the button
#           clear_output()
#           print('Something happens!')     

        # print(list(upload.value.values())[0])
        
        # csv_name = "./preds" + str(version["v"] - 1) + ".csv"
        csv_name = "https://salt-lab.herokuapp.com/preds.csv"
        data = pd.read_csv(csv_name, delimiter=",")


        # f = "./up" + str(version["v"] - 1) + ".mp4"
        f = "https://salt-lab.herokuapp.com/up.mp4"
        print(f)
        wid = Video(f)
        wid.layout.width='640px'
        wid._enable_keyboard=True


        av_coords = []
        angles = []
        colors = []
        magnitude = []


        for (v, a) in zip(data['valence'], data['arousal']):
          if(np.isnan(v) or np.isnan(a)):
            continue
          else:
            av_coords.append((v, a))
            magnitude.append(hypot(v, a))
            deg = (degrees(atan2(a, v)) + 360) % 360;
            angles.append(deg)

        for angle in angles:
          colors.append(hsv_to_rgb([angle/360, 1, 1]))

        fig, ax = plt.subplots()
        # x = np.arange(0, len(magnitude))
        (markers, stemlines, baseline) = plt.stem(magnitude, markerfmt=' ')
        plt.setp(stemlines, linestyle="-", color=colors, linewidth=0.5 )

        wid.display()

        @output.capture()
        def onclick(event):
#             print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
#                   ('double' if event.dblclick else 'single', event.button,
#                    event.x, event.y, event.xdata, event.ydata))
            wid.current_time = event.x/30

        cid = fig.canvas.mpl_connect('button_press_event', onclick)

        plt.show()
    
button.on_click(on_button_clicked)
widgets.VBox([upload, button, output])
# display(upload)